### MIRI Imaging reduction

Built on [Crab reduction toolkit](https://github.com/1054/Crab.Toolkit.JWST/)

for *JWST* program 3224

with additional intermediate steps by Leonid Sajkov

leonid.sajkov@tufts.edu

Tufts University, June 2024

**Notes**:
- Using this notebook requires downloading and installing the Crab reduction toolkit (linked above, or at: https://github.com/1054/Crab.Toolkit.JWST/). The notebook was built on toolkit version v20230109_before_huge_mosaic, downloaded May 22, 2024.
- Besides the Crab redution toolkit, this notebook also requries two pieces of code:
    - med_filt_and_sup_bkg_sub.py = code for median filering and super background subtraction;
    - post_reduction_steps.py = code for sorting reduced data and creating TA box cutouts.
- This notebook runs code in the terminal, using the subprocess run module.


In [ ]:
### 1
### Import necessary directories ###
import os
from subprocess import run
from IPython import display

from astroquery.mast import Observations

def simple_output(process, message):
    if (~save_outputs) & (process.returncode == 0):
        display.clear_output(wait = True)
        display.display(message)

In [ ]:
### 2
### Set necessary paths/parameters ###
# glbl_dir - for all final project files
# loc_dir - local directory for storing intermediate steps (files can get very large, so syncing them with cloud servers can take a while)
###

### SAVE LOGS: Y/N
# IMPORTANT: the text log outputs from this process are long and can take a lot of memory
# when running a notebook like this one locally, caching the logs (as many editors do automatically)
# can cause the editor to crash/slow down the entire process. Change to true to save output logs. 
save_outputs = False
### 

glbl_dir = 'jw3224_reduction'
loc_dir = 'loc_jw3224_reduction'

# set Calibration References Data System (CRDS) paths
CRDS_SERVER_URL = 'https://jwst-crds.stsci.edu'
CRDS_CONTEXT = 1234

# set paths for running Crab Toolkit
os.environ["PATH"] += os.pathsep + f'{glbl_dir}/code/crab_toolkit/bin'
os.environ["CRDS_PATH"] = f'{glbl_dir}/crds_cache'
os.environ["CRDS_SERVER_URL"] = CRDS_SERVER_URL
os.environ["CRDS_CONTEXT"] = f'jwst_{CRDS_CONTEXT}.pmap'

# set paths to raw data/working directory
raw_data_dir = f'{loc_dir}/raw_data/miri_imaging/jw*/jw*_uncal.fits'
working_dir = f'{loc_dir}/reducing_data'

In [ ]:
### 3
### Login and download data from MAST
# Input:
# - MAST login token
# - Program ID
# Output:
# - Raw, uncal files from the given program
###
     
program_id = '3224'

MAST_login_token = ''

if MAST_login_token != '':
    Observations.login(MAST_login_token)

os.chdir('/'.join(raw_data_dir.split('/')[:2]))
raw_download = run(f'go-jwst-query-by-program-id.py {program_id} --download')

simple_output(raw_download,
            f'raw uncal data downloaded to {raw_data_dir}')

In [ ]:
### 4
### Open working directory and copy uncal files ###
# Input:
# - raw data in raw_data_dir, of the form jw*/jw*_uncal.fits
# Output:
# - in working_dir, folders of the format jw*_mirimage (or jw*_gs*)
# - each folder contains an "uncals" folder with the associated uncal image
###

# open working directory (where intermediate reduction steps will be stored)
os.chdir(working_dir)

## copy uncal files to working directory
uncal_precopy = run(f'go-jwst-imaging-precopy-uncal-files {raw_data_dir}',
                    shell = True)

simple_output(uncal_precopy,
            f'uncal files copied to {working_dir}')

In [ ]:
### 5
### Precache necessary CRDS files
# only if files are not cached already #
# Output:
# - under glbl_dir/crds_cache, caches all the necessary files for the operation of the pipeline
###

precache_crds = run('go-jwst-imaging-precache-crds-reference-files', shell = True)

simple_output(precache_crds,
            f'precached CRDS files')

### Imaging data reduction

In [ ]:
### 6
### Stage 1: uncal -> rate ###
# runs step 1 of the Crab toolkit pipeline
# Input:
# - uncal files under the working directory, as created in cell #4
# Process:
# - runs step 1 of the standard Crab Toolkit pipeline, based on step 1 of the standard JWST pipeline
# Output:
# - in each subdirectory in the working directory, creates a calibrated1_rates folder
# - in this folder, places 6 files:
#    - jw*_mirimage_rate_calwebb_detector1.asdf
#    - jw*_mirimage_ramp.fits
#    - jw*_mirimage_rate_before_applying_flatfield
#    - jw*_mirimage_rate_applying_flatfield_flatfile.fits
#    - jw*_mirimage_rateints.fits
#    - jw*_mirimage_rate.fits - final rate file
###

uncal_to_rate = run('go-jwst-imaging-stage-1',
                    shell = True)

simple_output(uncal_to_rate,
            f'processed uncal to rate files')

In [ ]:
### 7
### Stage 2.1: rate -> cal
# runs stage 2 of the Crab toolkit pipeline
# Input:
# - rate files as produced in stage 1 (cell #6)
# - takes working_dir/jw*_mirimage/calibrated1_rates/jw*_mirimage_rate.fits
# User input:
# - SKIP_BKGSUB: skip uniform background subtraction step using skymatch. Additional subtraction will be done after stage 3, pass 1
# - DETECT_THRESH: SExtractor DETECT_THRESHold 
# Process:
# - runs stage 2 2 of the standard Crab Toolkit pipeline, based on stage 2 of the standard JWST pipeline
#   - step 1. standard JWST pipeline
#   - step 2. ()uses skymatch to subtract a uniform background (here skipped)
#   - step 3. deprecated/merged into stage 3
#   - step 4. correct astrometry
# Output:
# - in each subdirectory of the working directory, creates a calibrated2_cals folder:
# - in this folder, places 2 files and a subdirectory:
#   - jw*_mirimage_cal.fits - calibrated file
#   - jw*_mirimage_cal_cat_for_tweakreg.csv
#   - jw*_mirimage_cal_run_sextractor_classic_dir - contains SExtractor-related files, including the seg_map
###

SKIP_BKGSUB = True
DETECT_THRESH = 10
OVERWRITE = False

rate_to_cal = run('go-jwst-imaging-stage-2 ' +\
                  (' --overwrite' if OVERWRITE else "")+\
                  (' --no_bkgsub' if SKIP_BKGSUB else "") +\
                  f' --detect-thresh {DETECT_THRESH}',
                  shell = True)

[]

simple_output(rate_to_cal,
            f'processed rate to cal files')

In [ ]:
### OPTIONAL, but recommended BEFORE running intermediate steps
### make backups of cal files:
import fnmatch, shutil

for file in fnmatch.filter(os.listdir(working_dir), 'jw*_mirimage'):
    shutil.copy(f'{working_dir}/{file}/calibrated2_cals/{file}_cal.fits',
                f'{working_dir}/{file}/calibrated2_cals/{file}_cal_backup.fits')

In [ ]:
### 8 
### Intermediate steps
# performs custom steps in background reduction
# Input:
# - cal files as produced in stage 2 (cell #7)
# Settings:
# --median-filter/--no-median-filter = whether to perform median filering on the "hot" pixels in the image (default: True)
# --window-size = if performing median filtering, what size the "window" should be. Takes int values. (default: 1)
# --super-bkg-sub/--no-super-bkg-sub = whetehr to perform super background subtraction on the images (default: True)
# --small-source-thresh = remove small sources from the segmentation map, so that they will be considered in the 
#                         super median background. Use to avoid "hot" pixels and spurious detections clogging the background.
#                         If 0, does not remove small sources. Takes int values. (default: 0)
# --dilate-seg/--no-dilate-seg = whether to dilate the segmentation map when masking sources for super background.
# --dialte-iterations = how many iterations of the seg map dilation to perform. Takes int values. (default: 2)
# --median-thresh = minimum number of images in a FILTER/DURATION stack in order to perform subtraction.
#                   e.g.: if median-thresh = 5 and there are 3 exposures in F1500W lasting 88.8s, none of these images will have
#                   a super median background subtracted. Takes int values. (default: 5)
# Process:
# - median filters "hot" pixels in the image (labeled by nan in stage 2 of the pipeline)
#   - replaces the pixel with the median value of a square of lenght 2*window-size + 1 centered on the pixel
# - constructs a "median superstack", a dictionary containing a median background for each FILTER/DURATION pair
#   - e.g.: if there are exposures in F560W lasting 55.5s and 180.8s, as well as exposures in F1130W lasting 55.5s,
#           the median superstack will contain an entry for F560W_555, F560W_1808, and F1130W_555
# - subtracts the median background from each image, if there are enough exposures in the specific FILTER/DURATION pair
# Output:
# - in the calibrated2_cals directory, creates:
#   - _filt file: median-filtered cal files
#   - _sub file: super-median-background-subtracted filt files (or cal files, if --no-median-filter)
#   - _cal file: replaces the original cal file with the filtered and/or bkg-subtracted file
###

MEDIAN_FILTER = True
WINDOW_SIZE = 1
SUPER_BKG_SUB = True
SMALL_SOURCE_THRESH = 20
DILATE_SEG = True
DILATE_ITERATIONS = 2
MEDIAN_THRESH = 5

#optional: suppress default python warnings, as functions often encounter all-nan slices that they feel the need to warn us about
SUPPRESS_WARNINGS = True

intermediate_steps_cmdline = f'python {'-Wignore' if SUPPRESS_WARNINGS else ''} med_filt_and_sup_bkg_sub.py' +\
                             f' {"--median-filter" if MEDIAN_FILTER else "--no-median-filter"}' +\
                             f' --window-size {WINDOW_SIZE}' +\
                             f' {"--super-bkg-sub" if SUPER_BKG_SUB else "--no-super-bkg-sub"}' +\
                             f' --small-source-thresh {SMALL_SOURCE_THRESH}' +\
                             f' {"--dilate-seg" if DILATE_SEG else "--no-dilate-seg"}' +\
                             f' --dilate-iterations {DILATE_ITERATIONS}' +\
                             f' --median-thresh {MEDIAN_THRESH}'

intermediate_steps = run(intermediate_steps_cmdline,
                         shell = True)

In [ ]:
### 9
### Stage 2.2: cal -> i2d
# runs stage 3 of the Crab toolkit pipeline
# Input:
# - cal files as produced in stage 2 (cell #7)
# Ouptut:
# - mosaicked i2d files
###

cal_to_i2d_pass1 = run('go-jwst-imaging-stage-3 --detect-thresh 10 --pixel-scale-ratio 1',
                        shell = True)

simple_output(cal_to_i2d_pass1,
            f'processed cal files to i2d')

In [ ]:
### 10
### Package results/produce TA box cutouts
# Input:
# - i2d files from calibrated3_cals
# Arguments:
# destination_directory = where to place reduced, packaged files
# (optional): working_directory = the reducing_data folder, if not already specified in above cells
# Options:
# --date = date the reduction was done, for bookkeeping. Optional. Takes string. (default: '', defaults to today's date)
# --make-cutouts/--no-make-cutouts = make cutouts of the TA box? (produces both pdf and fits)
# Output:
# - reduced_data folder (destination directory) with subdirectories labeled by observation ID
# - in each subdirectory, creates additional subdirectories labeled by FILTER
#   - here, places the corresponding i2d file, TA box cutout as a fits file, and TA box cutout as a pdf
###

DESTINATION_DIRECTORY = 'loc_jw3224_reduction/reduced_data'
WORKING_DIRECTORY = ''
DATE = ''
MAKE_CUTOUTS = True

package_results_cmdline = f'post_reduction_steps.py {DESTINATION_DIRECTORY} ' +\
                          f' {WORKING_DIRECTORY}' +\
                          f' {"--date" + DATE if DATE != "" else ""}' +\
                          f' {"--make-cutouts" if MAKE_CUTOUTS else "--no-make-cutouts"}'

package_results = run(package_results_cmdline,
                      shell = True)

simple_output(package_results,
            f'final reduced data can be found in:\n{DESTINATION_DIRECTORY}')